In [26]:
import numpy as np 
import pandas as pd 

In [27]:
normaltr = pd.read_csv(r'C:\Users\ESPACE MULTIMEDIA\Desktop\Project security\normaltr_for_train.csv')
udp = pd.read_csv(r'C:\Users\ESPACE MULTIMEDIA\Desktop\Project security\udpflood_for_train.csv')
syn = pd.read_csv(r'C:\Users\ESPACE MULTIMEDIA\Desktop\Project security\synflood_for_train.csv')
icmp = pd.read_csv(r'C:\Users\ESPACE MULTIMEDIA\Desktop\Project security\icmpflood_for_train.csv')


In [28]:
udp1 = udp.drop(udp.index[5000:65537])
syn1 = syn.drop(syn.index[5000:124682])
icmp1 = icmp.drop(icmp.index[5000:70469])



In [29]:
normaltr['Label']=0
udp1['Label']=1
syn1['Label']=2
icmp1['Label']=3



In [30]:
data = pd.concat([normaltr,syn1,udp1,icmp1], axis = 0 )

In [31]:
data.shape

(16204, 84)

In [32]:
data2= data.drop(['Flow ID','Src IP','Dst IP','Timestamp','Src Port', 'Dst Port'],axis=1)
 
data2= data2[np.isfinite(data2).all(1)] 


In [33]:
data2.shape

(16154, 78)

In [34]:
x= data2.drop(['Label'],axis=1)
y= data2['Label']

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.8 , test_size=0.2)

In [37]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

In [38]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_train)

In [39]:
lasso = Lasso(alpha=0.01)
lasso.fit(X_train_std, y_train)


Lasso(alpha=0.01)

In [40]:
coefficients = lasso.coef_
selected_columns = X_train.columns[coefficients != 0]
inselected_columns = X_train.columns[coefficients == 0]

In [41]:
selected_columns

Index(['Protocol', 'TotLen Bwd Pkts', 'Bwd Pkt Len Max', 'Bwd Pkt Len Std',
       'Flow IAT Mean', 'Bwd IAT Min', 'Pkt Len Min', 'Pkt Len Max',
       'Pkt Len Var', 'SYN Flag Cnt', 'RST Flag Cnt', 'ACK Flag Cnt',
       'Subflow Bwd Byts', 'Init Bwd Win Byts', 'Active Mean', 'Active Max',
       'Idle Std'],
      dtype='object')

In [42]:
from sklearn.tree import DecisionTreeClassifier

In [43]:
clf_gini = DecisionTreeClassifier(criterion='gini', random_state=0)
clf_gini.fit(X_train[selected_columns], y_train)

DecisionTreeClassifier(random_state=0)

In [44]:
y_pred = clf_gini.predict(X_test[selected_columns])


In [58]:
from sklearn.metrics import accuracy_score ,f1_score , recall_score,confusion_matrix


In [59]:
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred,average='weighted')
f1 = f1_score(y_test, y_pred,average='weighted')
cm=confusion_matrix(y_test, y_pred)

In [60]:
print("L'accuracy est de :", accuracy)
print("Le recall est de :", recall)
print("Le F1-score est de :", f1)
cm

L'accuracy est de : 0.9993809965954813
Le recall est de : 0.9993809965954813
Le F1-score est de : 0.9993820115965609


array([[ 233,    0,    0,    0],
       [   0, 1012,    0,    0],
       [   0,    0,  979,    0],
       [   2,    0,    0, 1005]], dtype=int64)

In [48]:
def predict(predicion):
        if predicion == 0:
                return 'Normal Traffic'
        elif predicion == 1:
                return 'Udp flood'
        elif predicion == 2:
                return 'Syn flood'
        else :
                return 'icmp flood'

In [66]:
#t = pd.read_csv(r'C:\Users\ESPACE MULTIMEDIA\Desktop\Project security\synflood_to_predict.csv') # for syn flood
t= pd.read_csv(r'C:\Users\ESPACE MULTIMEDIA\Desktop\Project security\udp_to_predict.csv') #for udp flood
#t= pd.read_csv(r'C:\Users\ESPACE MULTIMEDIA\Desktop\Project security\icmp_to_predict.csv') #for icmp flood
t.shape

(65537, 84)

In [67]:
indices = t.columns.get_indexer(selected_columns,)
values = t.iloc[1500, indices].values
values


array([17, 0.0, 0.0, 0.0, 2125282.212121212, 0, 0.0, 0.0, 0.0, 0, 0, 0, 0,
       -1, 0.0, 0.0, 0.0], dtype=object)

In [68]:
predicion = clf_gini.predict(np.array(values.reshape(1,-1)))[0]
predict(predicion)



c:\Users\ESPACE MULTIMEDIA\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


'Udp flood'